# SQL

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adamelliotfields/jupyter/blob/main/notebooks/sql.ipynb)
[![Render nbviewer](https://raw.githubusercontent.com/jupyter/design/main/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/adamelliotfields/jupyter/blob/main/notebooks/sql.ipynb)

In [1]:
%load_ext sql

# https://jupysql.ploomber.io/en/latest/api/configuration.html#options
%config SqlMagic.autopandas = False
%config SqlMagic.displaycon = False
%config SqlMagic.displaylimit = 20
%config SqlMagic.feedback = 0

%sql duckdb://


In [2]:
%%bash
# check for $KAGGLE_USERNAME AND $KAGGLE_KEY
if [ -z "$KAGGLE_USERNAME" ] ; then
  echo "error: KAGGLE_USERNAME is not set"
  exit 0
fi

if [ -z "$KAGGLE_KEY" ] ; then
  echo "error: KAGGLE_KEY is not set"
  exit 0
fi

# download the Sakila dataset from Kaggle
# https://motherduck.com/blog/analyze-sqlite-databases-duckdb
rm -f sqlite-sakila-sample-database.zip sqlite-sakila.db README.txt 'SQLite3 Sakila Sample Database ERD.png'
rm -rf sqlite-sakila-db/
poetry run kaggle datasets download --force --unzip -d atanaskanev/sqlite-sakila-sample-database


100%|██████████| 4.29M/4.29M [00:00<00:00, 5.52MB/s]


In [3]:
import duckdb
conn = duckdb.connect()
# native Python DBAPI connection bypasses SQLAlchemy (works better in Jupyter)
# this prevents `%sqlcmd` commands from working as they are provided by SQLAlchemy
%sql conn


In [4]:
%%sql
-- load the SQLite extension for DuckDB
INSTALL sqlite;
LOAD sqlite;
-- normally duckdb attempts to infer the type of the columns from sqlite
-- set this to `true` to instruct duckdb to treat all sqlite columns as `VARCHAR`
-- https://duckdb.org/docs/extensions/sqlite_scanner.html#data-types
SET GLOBAL sqlite_all_varchar=false;
CALL sqlite_attach('sqlite-sakila.db');


Success


In [5]:
%%sql
-- SELECT * FROM information_schema.tables;
SHOW TABLES;


name
actor
address
category
city
country
customer
customer_list
film
film_actor
film_category


In [6]:
%%sql
-- SELECT column_name, column_default, is_nullable FROM information_schema.columns WHERE table_name = 'actor';
DESCRIBE actor;


column_name,column_type,null,key,default,extra
actor_id,DOUBLE,YES,None,None,None
first_name,VARCHAR,YES,None,None,None
last_name,VARCHAR,YES,None,None,None
last_update,TIMESTAMP,YES,None,None,None


In [7]:
%%sql
-- show total by category
SELECT c.name category_name, count(*) total
FROM rental r
LEFT JOIN inventory i USING (inventory_id)
LEFT JOIN film_category fc USING (film_id)
LEFT JOIN category c USING (category_id)
GROUP BY category_name
ORDER BY total DESC;


category_name,total
Sports,1179
Animation,1166
Action,1112
Sci-Fi,1101
Family,1096
Drama,1060
Documentary,1050
Foreign,1033
Games,969
Children,945


In [8]:
%%sql
-- show revenue by category
SELECT c.name category_name, SUM(IFNULL(p.amount, 0)) revenue
FROM category c
LEFT JOIN film_category fc ON c.category_id = fc.category_id
LEFT JOIN film f ON fc.film_id = f.film_id
LEFT JOIN inventory i ON f.film_id = i.film_id
LEFT JOIN rental r ON i.inventory_id = r.inventory_id
LEFT JOIN payment p ON r.rental_id = p.rental_id
GROUP BY category_name
ORDER BY revenue DESC;


category_name,revenue
Sports,5314.209999999848
Sci-Fi,4756.979999999872
Animation,4656.299999999864
Drama,4587.389999999875
Comedy,4383.579999999896
Action,4375.849999999871
New,4351.619999999898
Games,4281.329999999893
Foreign,4270.66999999989
Family,4226.069999999881
